In [1]:
import pandas as pd 
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pymongo

In [2]:
#Browser setup
lp_url = "https://www.lonelyplanet.com/best-in-travel/countries"
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome")
browser.visit(lp_url)

In [3]:
lp_soup = bs(browser.html, 'lxml')

In [4]:
results = lp_soup.find_all("h1")

In [5]:
top_countries = [result.get_text(strip=True).split('.') for result in results]
df = pd.DataFrame(top_countries)
df.columns=['rank', 'country']
df = df.iloc[1:,:]

In [6]:
df['city'] = np.nan
df['source'] = "lonelyplanet"
df.columns= ['rank', 'city','country','source']

In [7]:
lp_df = df
lp_df

,rank,city,country,source
1,1,Sri Lanka,NaN,lonelyplanet
2,2,Germany,NaN,lonelyplanet
3,3,Zimbabwe,NaN,lonelyplanet
4,4,Panama,NaN,lonelyplanet
5,5,Kyrgyzstan,NaN,lonelyplanet
6,6,Jordan,NaN,lonelyplanet
7,7,Indonesia,NaN,lonelyplanet
8,8,Belarus,NaN,lonelyplanet
9,9,São Tomé & Príncipe,NaN,lonelyplanet
10,10,Belize,NaN,lonelyplanet


In [8]:
ins_url='https://www.thisisinsider.com/the-20-most-popular-travel-destinations-in-the-world-2016-9'

In [9]:
response = requests.get(ins_url)

In [10]:
ins_soup = bs(response.text, 'html.parser')
results = ins_soup.find_all('h2', class_="slide-title-text")

In [11]:
data = []
for result in results:
    data.append(result.get_text(strip=True).replace('. ', ', ').split(","))

In [12]:
ins_df = pd.DataFrame(data)

In [13]:
ins_df.loc[8]=["12","Barcelona", "Spain"]
ins_df.loc[7]="13","Amsterdam", "Netherlands"
ins_df.loc[14]="6","Singapore", "Singapore"
ins_df.loc[15]="5","New York", "United States of America"

In [14]:
ins_df.columns=['rank','city','country']
ins_df['source'] = 'insidertravel'

In [15]:
ins_df[["rank"]] = ins_df[["rank"]].apply(pd.to_numeric)
ins_df= ins_df.sort_values(by="rank", ascending=True)
ins_df.reset_index(drop=True)

,rank,city,country,source
0,1,Bangkok,Thailand,insidertravel
1,2,London,England,insidertravel
2,3,Paris,France,insidertravel
3,4,Dubai,United Arab Emirates,insidertravel
4,5,New York,United States of America,insidertravel
5,6,Singapore,Singapore,insidertravel
6,7,Kuala Lumpur,Malaysia,insidertravel
7,8,Istanbul,Turkey,insidertravel
8,9,Tokyo,Japan,insidertravel
9,10,Seoul,South Korea,insidertravel


In [16]:
# grabbed the states
url='https://alphabetizer.flap.tv/lists/list-of-states-in-alphabetical-order.php'
browser.visit(url)
html=browser.html
soup=bs(html,'lxml')

In [17]:
us_states=[]
results=soup.find_all('li')
for x in range(len(results)):
    us_states.append(results[x].get_text())

In [18]:
#cleaned up data
url='https://www.tripadvisor.com/TravelersChoice-Destinations-cTop-g1'

browser.visit(url)
html=browser.html
soup=bs(html,"html.parser")
rating=[]
city=[]
country=[]
id=[]
results=soup.find_all('div',class_='winnerLayer')
for result in results:
    rating.append(result.div.span.get_text())
    place=result.find('div',class_='winnerName').div.a.get_text()
    if len(place.split(',')) ==2:
        city.append(place.split(',')[0])
        country.append(place.split(',')[1])
    else:
        city.append(np.nan)
        country.append(place.split(',')[0])

for x in range(len(country)):
    if (country[x].strip() in us_states):
        country[x]='United States'
        if city[x]==np.nan:
            city[x]=country[x] 
    
ta_df=pd.DataFrame(dict(rank=rating,city=city,country=country,source="trip_advisor"))

In [19]:
#cleaned up data
url="https://www.travelandleisure.com/trip-ideas/best-places-to-travel-in-2018"
browser.visit(url)
html=browser.html
soup=bs(html,'lxml')

In [20]:
country=[]
city=[]
counter=0
results=soup.find_all('h3',class_='glide-slide-title')
check_for_doubles=""
for result in results:
    response=result.get_text()
    
    if check_for_doubles != response:
        if 'The 50 Best Places' not in response:

            response_split=response.strip().split(",")

            if len(response_split)==2:

                # two places added together with 'and' in country
                first_part=response_split[0].strip()
                second_part=response_split[1].strip()

                if ' and' in second_part:
                    
                    second_part=second_part.split(' and')[0]
                    
                country.append(second_part)   
                city.append(first_part)
            else:
                country.append(response_split[0].strip())
                city.append(np.nan)
                
        
        check_for_doubles=response
for x in range(len(country)):
    if (country[x] in us_states):
        country[x]='United States'
        if city[x]==np.nan:
            city[x]=country[x]
       
        
rating=[x for x in range(1,len(country)+1)]
tl_df=pd.DataFrame(dict(rank=rating,city=city,country=country,source="travel and leisure"))
tl_df

,rank,city,country,source
0,1,Šolta,Croatia,travel and leisure
1,2,Greenville,United States,travel and leisure
2,3,NaN,Grenada,travel and leisure
3,4,Buenos Aires,Argentina,travel and leisure
4,5,Los Cabos,Mexico,travel and leisure
5,6,Walla Walla Valley,United States,travel and leisure
6,7,NaN,Uzbekistan,travel and leisure
7,8,NaN,Egypt,travel and leisure
8,9,Marrakesh,Morocco,travel and leisure
9,10,NaN,Fiji,travel and leisure


In [21]:
frames = [lp_df,ins_df,tl_df, ta_df]
results = pd.concat(frames, sort=True)

In [22]:
travel_data = results.reset_index(drop=True)

In [23]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [24]:
# Define database and collection
db = client.travel_db
collection = db.items

In [25]:
def transform_data(row):
    data = row.to_dict()
    collection.insert_one(data)

In [26]:
#df = df.reset_index()
travel_data.apply(transform_data, axis=1) 

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
       ... 
125    None
126    None
127    None
128    None
129    None
130    None
131    None
132    None
133    None
134    None
135    None
136    None
137    None
138    None
139    None
140    None
141    None
142    None
143    None
144    None
145    None
146    None
147    None
148    None
149    None
150    None
151    None
152    None
153    None
154    None
Length: 155, dtype: object